In [1]:
import pandas as pd
import sqlite3
from IPython.display import HTML
%matplotlib inline
import tensorflow as tf
import numpy as np

In [28]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val,val)

In [2]:
conn = sqlite3.connect('papers.db')
test = pd.read_sql_query('SELECT * FROM papers', conn)

In [3]:
target = pd.read_csv('ipfjes-case-control-studies.csv', usecols=['pmid','title','firstauthor','lastauthor',
                                                             'journal','pubdate','pubtype',
                                                             'abstract', 'keywords', 'rcr', 'citedby', 'cites'])

In [4]:
target['target'] = 1

test['target'] = 0

training = pd.concat([target, test.head(14)])

from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(training['abstract'].values)

from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
targets = training['target'].values
classifier.fit(counts, targets)

test_counts = count_vectorizer.transform(test['abstract'].values)
predictions = classifier.predict(test_counts)
predictions_prob = classifier.predict_proba(test_counts)

In [41]:
test['testresult'] = predictions
test['testprob'] = pd.DataFrame(predictions_prob)[1]
test['link'] = 'https://www.ncbi.nlm.nih.gov/pubmed/' + test['pmid']

In [49]:
results = test[(test.testresult == 1) & (test.date.str.contains('2017'))].sort_values(by='testprob', ascending=False)
results[['title', 'link']].style.format(make_clickable) # clickable results are important

In [50]:
results.head()

,pmid,author,date,title,journal,key_words,pub_type,abstract,target,testresult,testprob,link
1510,28182861,"Egashira R, Jacob J, Kokosi MA, Brun AL, Rice ...",2017 Jul,Diffuse Pulmonary Ossification in Fibrosing In...,Radiology,"Aged, Biopsy, Female, Humans, Idiopathic Pulmo...",Journal Article,Purpose To investigate the prevalence of diffu...,0,1,1.000000,https://www.ncbi.nlm.nih.gov/pubmed/28182861
4560,27471881,"Mlika M, Kria N, Braham E, Chebbi C, El Mezni F",2017,The diagnostic importance of the bronchoalveol...,Journal of immunoassay & immunochemistry,"Adolescent, Adult, Aged, Bronchoalveolar Lavag...",Journal Article,Multidisciplinary concertation is mandatory in...,0,1,0.999997,https://www.ncbi.nlm.nih.gov/pubmed/27471881
3244,28130491,"Podolanczuk AJ, Raghu G, Tsai MY, Kawut SM, Pe...",2017 May,"Cholesterol, lipoproteins and subclinical inte...",Thorax,"Aged, Aged, 80 and over, Biomarkers/blood, Cho...","Letter, Multicenter Study",We investigated associations of plasma lipopro...,0,1,0.999995,https://www.ncbi.nlm.nih.gov/pubmed/28130491
86,27708114,"Kreuter M, Bonella F, Maher TM, Costabel U, Sp...",2017 Feb,Effect of statins on disease-related outcomes ...,Thorax,"Adult, Aged, Aged, 80 and over, Disease Progre...","Clinical Trial, Phase III, Journal Article, Ra...",BACKGROUND: Data are conflicting regarding the...,0,1,0.829780,https://www.ncbi.nlm.nih.gov/pubmed/27708114
99,28115263,"Moradi P, Keyvani H, Javad Mousavi SA, Karbala...",2017 Mar,Investigation of viral infection in idiopathic...,Microbial pathogenesis,"Adenoviruses, Human/*isolation & purification,...",Journal Article,AIM OF THE STUDY: Idiopathic pulmonary fibrosi...,0,1,0.809333,https://www.ncbi.nlm.nih.gov/pubmed/28115263


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import numpy as np
import pandas
from sklearn import metrics
import tensorflow as tf

FLAGS = None

MAX_DOCUMENT_LENGTH = 10
EMBEDDING_SIZE = 50
n_words = 0
MAX_LABEL = 15
WORDS_FEATURE = 'words' # Name of the input words feature.

In [4]:
def estimator_spec_for_softmax_classification(
    logits, labels, mode):
  """Returns EstimatorSpec instance for softmax classification."""
  predicted_classes = tf.argmax(logits, 1)
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions={
            'class': predicted_classes,
            'prob': tf.nn.softmax(logits)
        })

  onehot_labels = tf.one_hot(labels, MAX_LABEL, 1, 0)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

  eval_metric_ops = {
      'accuracy': tf.metrics.accuracy(
          labels=labels, predictions=predicted_classes)
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [5]:
def bag_of_words_model(features, labels, mode):
  """A bag-of-words model. Note it disregards the word order in the text."""
  bow_column = tf.feature_column.categorical_column_with_identity(
      WORDS_FEATURE, num_buckets=n_words)
  bow_embedding_column = tf.feature_column.embedding_column(
      bow_column, dimension=EMBEDDING_SIZE)
  bow = tf.feature_column.input_layer(
      features,
      feature_columns=[bow_embedding_column])
  logits = tf.layers.dense(bow, MAX_LABEL, activation=None)

  return estimator_spec_for_softmax_classification(
      logits=logits, labels=labels, mode=mode)



In [6]:
def rnn_model(features, labels, mode):
  """RNN model to predict from sequence of words to a class."""
  # Convert indexes of words into embeddings.
  # This creates embeddings matrix of [n_words, EMBEDDING_SIZE] and then
  # maps word indexes of the sequence into [batch_size, sequence_length,
  # EMBEDDING_SIZE].
  word_vectors = tf.contrib.layers.embed_sequence(
      features[WORDS_FEATURE], vocab_size=n_words, embed_dim=EMBEDDING_SIZE)

  # Split into list of embedding per word, while removing doc length dim.
  # word_list results to be a list of tensors [batch_size, EMBEDDING_SIZE].
  word_list = tf.unstack(word_vectors, axis=1)

  # Create a Gated Recurrent Unit cell with hidden size of EMBEDDING_SIZE.
  cell = tf.contrib.rnn.GRUCell(EMBEDDING_SIZE)

  # Create an unrolled Recurrent Neural Networks to length of
  # MAX_DOCUMENT_LENGTH and passes word_list as inputs for each unit.
  _, encoding = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)

  # Given encoding of RNN, take encoding of last step (e.g hidden size of the
  # neural network of last step) and pass it as features for softmax
  # classification over output classes.
  logits = tf.layers.dense(encoding, MAX_LABEL, activation=None)
  return estimator_spec_for_softmax_classification(
      logits=logits, labels=labels, mode=mode)


In [9]:
def main(unused_argv):
  global n_words
  # Prepare training and testing data
  dbpedia = tf.contrib.learn.datasets.load_dataset(
      'dbpedia', test_with_fake_data=FLAGS.test_with_fake_data)
  x_train = pandas.DataFrame(dbpedia.train.data)[1]
  y_train = pandas.Series(dbpedia.train.target)
  x_test = pandas.DataFrame(dbpedia.test.data)[1]
  y_test = pandas.Series(dbpedia.test.target)

  # Process vocabulary
  vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(
      MAX_DOCUMENT_LENGTH)

  x_transform_train = vocab_processor.fit_transform(x_train)
  x_transform_test = vocab_processor.transform(x_test)

  x_train = np.array(list(x_transform_train))
  x_test = np.array(list(x_transform_test))

  n_words = len(vocab_processor.vocabulary_)
  print('Total words: %d' % n_words)

  # Build model
  # Switch between rnn_model and bag_of_words_model to test different models.
  model_fn = rnn_model
  if FLAGS.bow_model:
    # Subtract 1 because VocabularyProcessor outputs a word-id matrix where word
    # ids start from 1 and 0 means 'no word'. But
    # categorical_column_with_identity assumes 0-based count and uses -1 for
    # missing word.
    x_train -= 1
    x_test -= 1
    model_fn = bag_of_words_model
  classifier = tf.estimator.Estimator(model_fn=model_fn)

  # Train.
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={WORDS_FEATURE: x_train},
      y=y_train,
      batch_size=len(x_train),
      num_epochs=None,
      shuffle=True)
  classifier.train(input_fn=train_input_fn, steps=100)

  # Predict.
  test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={WORDS_FEATURE: x_test},
      y=y_test,
      num_epochs=1,
      shuffle=False)
  predictions = classifier.predict(input_fn=test_input_fn)
  y_predicted = np.array(list(p['class'] for p in predictions))
  y_predicted = y_predicted.reshape(np.array(y_test).shape)

  # Score with sklearn.
  score = metrics.accuracy_score(y_test, y_predicted)
  print('Accuracy (sklearn): {0:f}'.format(score))

  # Score with tensorflow.
  scores = classifier.evaluate(input_fn=test_input_fn)
  print('Accuracy (tensorflow): {0:f}'.format(scores['accuracy']))


if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--test_with_fake_data',
      default=False,
      help='Test the example code with fake data.',
      action='store_true')
  parser.add_argument(
      '--bow_model',
      default=False,
      help='Run with BOW model instead of RNN.',
      action='store_true')
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Total words: 7552
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_tf_random_seed': 1, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpo3u9_ym7', '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpo3u9_ym7/model.ckpt.
INFO:tensorflow:step = 1, loss = 2.7089
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmpo3u9_ym7/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0007447.
INFO:tensorflow:Restoring parameters from /tmp/tmpo3u9_ym7/model.ckpt-100
Accuracy (sklearn): 0.228571
INFO:tensorflow:Starting evaluation at 2017-07-17-20:15:09
INFO:tensorflow:Restoring parameters from /tmp/tmpo3u9_ym7/model.ckpt-100
INFO:tensorflow:Finished evaluation at 2017-07-17-20:15:09
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.228571, global_step = 100, loss

SystemExit: 

/home/drcjar/.virtualenvs/litsearch2/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
main('On Nov 9, it’s been an official 1 year since TensorFlow released. Looking back there has been a lot of progress done towards making TensorFlow the most used machine learning framework.')

Total words: 7552
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_tf_random_seed': 1, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpgrt7zkxf', '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpgrt7zkxf/model.ckpt.
INFO:tensorflow:step = 1, loss = 2.70871
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmpgrt7zkxf/model.ckpt.
INFO:tensorflow:Loss for final step: 0.000722945.
INFO:tensorflow:Restoring parameters from /tmp/tmpgrt7zkxf/model.ckpt-100
Accuracy (sklearn): 0.257143
INFO:tensorflow:Starting evaluation at 2017-07-17-20:19:06
INFO:tensorflow:Restoring parameters from /tmp/tmpgrt7zkxf/model.ckpt-100
INFO:tensorflow:Finished evaluation at 2017-07-17-20:19:06
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.257143, global_step = 100, l